# Анализ данных сервиса для чтения книг

## Цели исследования

После покупки сервиса по чтению книг, нам досталась и база данных, которую нужно изучить. 

В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. 

Информацию которую мы получим, можно будет в дальнейшем использовать для создания идеи нового продукта. 

## План задач

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Начало исследования

### Подготовка среды для работы с запросами SQL

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 


### Обзор таблиц

In [2]:
q = "SELECT * FROM books LIMIT 5"
q_2 = "SELECT * FROM ratings LIMIT 5"
q_3 = "SELECT * FROM reviews LIMIT 5"
q_4 = "SELECT * FROM publishers LIMIT 5"
q_5 = "SELECT * FROM books LIMIT 5"

In [3]:
books = pd.io.sql.read_sql(q, con = engine)      
ratings = pd.io.sql.read_sql(q_2, con = engine)
reviews = pd.io.sql.read_sql(q_3, con = engine)
publishers = pd.io.sql.read_sql(q_4, con = engine)
authors = pd.io.sql.read_sql(q_5, con = engine)

In [4]:
for i in [books, ratings, reviews, publishers, authors]:
    display(i.head())

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Задание 1. Сколько книг вышло после 1 января 2000 года?

In [5]:
query  = "SELECT COUNT(book_id) FROM books WHERE publication_date > TO_DATE('2000-01-01', 'YYYY-MM-DD')"

In [6]:
count_books = pd.io.sql.read_sql(query, con = engine)
count_books = count_books.reset_index()
print(f'После 1 января 2000 года вышло {(count_books.iloc[0,1])} книг')


После 1 января 2000 года вышло 819 книг


### Задание 2. Для каждой книги посчитать количество обзоров и среднюю оценку

In [7]:
query_2 = """
SELECT b.title, COUNT(DISTINCT rw.review_id), ROUND(AVG(r.rating), 1) rating_mean
FROM books b
LEFT JOIN ratings r ON b.book_id = r.book_id
LEFT JOIN reviews rw ON b.book_id = rw.book_id
GROUP BY b.title
ORDER BY COUNT(rw.review_id) DESC, rating_mean DESC
"""


In [8]:
review_rating = pd.io.sql.read_sql(query_2, con = engine)
review_rating

,title,count,rating_mean
0,Twilight (Twilight #1),7,3.7
1,The Hobbit or There and Back Again,6,4.1
2,The Catcher in the Rye,6,3.8
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.3
...,...,...,...
994,Essential Tales and Poems,0,4.0
995,Disney's Beauty and the Beast (A Little Golden...,0,4.0
996,Leonardo's Notebooks,0,4.0
997,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7


### Задание 3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц

In [9]:
query_3 = """
WITH book_counts AS (
    SELECT p.publisher, COUNT(b.book_id) as num_books
    FROM publishers p
    JOIN books b ON p.publisher_id = b.publisher_id
    WHERE b.num_pages > 50
    GROUP BY p.publisher
)
SELECT publisher, num_books
FROM book_counts
ORDER BY num_books DESC
LIMIT 1

"""

In [10]:
publisher = pd.io.sql.read_sql(query_3, con = engine)
publisher

,publisher,num_books
0,Penguin Books,42


### Задание 4. Определить автора с самой высокой средней оценкой книг — учитывать только книги с 50 и более оценками

In [11]:
query_4 = """
SELECT a.author, AVG(r.rating) avg_rating
FROM authors a
JOIN books b ON a.author_id = b.author_id
JOIN ratings r ON b.book_id = r.book_id
WHERE b.book_id IN
   (SELECT b.book_id
   FROM books b
   JOIN ratings  r ON b.book_id=r.book_id
   GROUP BY b.book_id
   HAVING count(r.rating_id)> 49)
GROUP BY a.author
ORDER BY avg_rating DESC
LIMIT 1
"""

In [12]:
author = pd.io.sql.read_sql(query_4, con = engine)
author

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


In [13]:
pd.io.sql.read_sql("SELECT book_id, COUNT(review_id) FROM reviews GROUP BY book_id ORDER BY COUNT(review_id) DESC ", con = engine)  

,book_id,count
0,948,7
1,656,6
2,497,6
3,963,6
4,696,6
...,...,...
989,599,1
990,187,1
991,156,1
992,984,1


### Задание 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
query_5 = """
WITH t AS 
(SELECT username, COUNT(review_id) cnt_id
FROM reviews
WHERE username IN (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) >50)
GROUP BY username)

SELECT ROUND(AVG(cnt_id), 1)
FROM t

"""

In [15]:
review = pd.io.sql.read_sql(query_5, con = engine)
review
print(f'В среднем количество обзоров у пользователей, которые поставили больше 50 оценок - {(review.iloc[0,0])}')

В среднем количество обзоров у пользователей, которые поставили больше 50 оценок - 24.3


## Выводы

### Той информации, которую мы получили из таблиц недостаточно для того, чтобы сделать полноценные выводы. 
Следует более глубоку изучить данные, в них есть ещё потенциал.

Что удалось найти:

- После 1 январе 2000 года вышло 819 книг. Много это или мало, не совсем понятно. Нужно понимать до какого года включительно у нас есть данные и возможно стоило бы посмотреть на распределение по годам.
- Количество обзоров на книги имеют разбор от 0 до 7.  
- Издательтво которое выпустило больше всего книг толще 50 страниц Penguin Books — британское издательство, основанное в 1935 году в Лондоне сэром Алленом Лейном и его братьями Ричардом и Джоном. Главной заслугой издательства считают демократизацию книжного рынка, превратившую книгу из предмета роскоши в удовольствие, доступное массам. Возможно количество выпущенных книг связано с возрастом самого издательства.
- Из популярных книг, которые получили свыше 50 оценок, наивысшию среднюю оценку в 4.3 получили книги J.K. Rowling/Mary GrandPré - британская писательница, сценаристка и кинопродюсер, наиболее известная как автор серии романов о Гарри Поттере Можно делать выводы о том, что нравится аудитории, например какой жанр и т д.
- сложно сказать, что-то о количестве обзоров. Но в любом случае, обзоры популяризируют чтение книг, а для сервиса эта метрика думаю будет важна. Можно найти способы её повышать. 